# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5d3c361e20>
├── 'a' --> tensor([[-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429]])
└── 'x' --> <FastTreeValue 0x7f5d3c361f40>
    └── 'c' --> tensor([[-0.0389, -0.0852, -2.0472, -1.0704],
                        [-0.4161,  0.9798,  0.5030, -0.2292],
                        [-2.2833,  0.4552, -0.3767,  0.7829]])

In [4]:
t.a

tensor([[-0.6719,  0.1360,  0.5594],
        [ 0.3111,  1.4062, -1.0429]])

In [5]:
%timeit t.a

62.5 ns ± 0.0695 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5d3c361e20>
├── 'a' --> tensor([[-1.0375, -1.3922,  2.0296],
│                   [ 1.0804,  0.7428, -0.5160]])
└── 'x' --> <FastTreeValue 0x7f5d3c361f40>
    └── 'c' --> tensor([[-0.0389, -0.0852, -2.0472, -1.0704],
                        [-0.4161,  0.9798,  0.5030, -0.2292],
                        [-2.2833,  0.4552, -0.3767,  0.7829]])

In [7]:
%timeit t.a = new_value

61.3 ns ± 0.0431 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6719,  0.1360,  0.5594],
               [ 0.3111,  1.4062, -1.0429]]),
    x: Batch(
           c: tensor([[-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829]]),
       ),
)

In [10]:
b.a

tensor([[-0.6719,  0.1360,  0.5594],
        [ 0.3111,  1.4062, -1.0429]])

In [11]:
%timeit b.a

58.1 ns ± 0.0326 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7318,  0.3256,  1.2091],
               [ 0.3324, -0.5042,  0.5991]]),
    x: Batch(
           c: tensor([[-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.044 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

849 ns ± 33.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 35.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 341 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 374 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5c90011580>
├── 'a' --> tensor([[[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]],
│           
│                   [[-0.6719,  0.1360,  0.5594],
│                    [ 0.3111,  1.4062, -1.0429]]])
└── 'x' --> <FastTreeValue 0x7f5c90011310>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 69.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5c96c4e910>
├── 'a' --> tensor([[-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429],
│                   [-0.6719,  0.1360,  0.5594],
│                   [ 0.3111,  1.4062, -1.0429]])
└── 'x' --> <FastTreeValue 0x7f5c96c651c0>
    └── 'c' --> tensor([[-0.0389, -0.0852, -2.0472, -1.0704],
                        [-0.4161,  0.9798,  0.5030, -0.2292],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 45.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 79.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0389, -0.0852, -2.0472, -1.0704],
                       [-0.4161,  0.9798,  0.5030, -0.2292],
                       [-2.2833,  0.4552, -0.3767,  0.7829]],
              
                      [[-0.0389, -0.0852, -2.0472, -1.0704],
                       [-0.4161,  0.9798,  0.5030, -0.2292],
                       [-2.2833,  0.4552, -0.3767,  0.7829]],
              
                      [[-0.0389, -0.0852, -2.0472, -1.0704],
                       [-0.4161,  0.9798,  0.5030, -0.2292],
                       [-2.2833,  0.4552, -0.3767,  0.7829]],
              
                      [[-0.0389, -0.0852, -2.0472, -1.0704],
                       [-0.4161,  0.9798,  0.5030, -0.2292],
                       [-2.2833,  0.4552, -0.3767,  0.7829]],
              
                      [[-0.0389, -0.0852, -2.0472, -1.0704],
                       [-0.4161,  0.9798,  0.5030, -0.2292],
                       [-2.2833,  0.4552, -0.3767,  0.7829]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829],
                      [-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829],
                      [-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829],
                      [-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829],
                      [-0.0389, -0.0852, -2.0472, -1.0704],
                      [-0.4161,  0.9798,  0.5030, -0.2292],
                      [-2.2833,  0.4552, -0.3767,  0.7829],
                      [-0.0389, -0.0852, -2.0472, -1.0704],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
